## Searching for bouts for a day of alsa recording

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-08-06 12:49:15,353 root         INFO     Running on pakhi


In [2]:
from ceciestunepipe.file import bcistructure as et

### Get the file locations for a session (day) of recordings

In [32]:
reload(et)
sess_par = {'bird': 's_b1253_21',
           'sess': '2021-07-31'}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='alsa')

raw_folder = exp_struct['folders']['alsa']
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [33]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1253_21',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-31',
 'alsa': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-31/alsa',
 'kwik': '/scratch/earneodo/s_b1253_21/alsa/kwik/2021-07-31',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1253_21/2021-07-31/alsa',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-31/alsa',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1253_21/alsa/msort/2021-07-31',
 'ksort': '/scratch/earneodo/s_b1253_21/alsa/ksort/2021-07-31'}

### read a session's bouts pandas
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [34]:
from ceciestunepipe.util.sound import boutsearch as bs

In [35]:
### load the search/spectrogram parameters
### these functions are necessary to load the pickle files, which is bad.
### need to not write modules/functions in the pickle

# function for getting one channel out of a wave file
def read_wav_chan(wav_path: str, chan_id: int=0) -> tuple:
    s_f, x = wavfile.read(wav_path, mmap=True)
    return s_f, x[:, chan_id]

def sess_file_id(f_path):
    n = int(os.path.split(f_path)[1].split('-')[-1].split('.wav')[0])
    return n

hparams_file_path = os.path.join(bouts_folder, 'bout_search_params.pickle')
with open(hparams_file_path, 'rb') as fh:
    hparams = pickle.load(fh)

In [36]:
#bpd, x, p = bs.get_bouts_in_file(one_wav_path, hparams)

bouts_auto_file_path = os.path.join(bouts_folder, hparams['bout_auto_file'])

#load. It is important to reset index because the manual curation requires unique indexing
bpd = pd.read_pickle(bouts_auto_file_path).reset_index(drop=True)
bpd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform
0,1207790,1215880,57973920,58362240,"[0.8444742589447913, 35.9684823423289, 24.9154...",1.629499,81.583,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,8090,"[[1207790, 1207835], [1208385, 1208415], [1208...",14,"[1, 8, 21, 119, 184, 191, 313, 353, 365, 382, ...",49,165.102041,"[-8, -9, -8, -8, -8, -8, -9, -9, -9, -9, -9, -..."


###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [37]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [38]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)

cleanup(bpd)

#### compute the spectrograms

In [39]:
bpd['spectrogram'] = bpd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + bouts_auto_file_path)
bpd.to_pickle(bouts_auto_file_path)

2021-08-06 12:57:48,133 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-31/alsa/bouts_ceciestunepipe/bout_auto.pickle


In [40]:
bpd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram
0,1207790,1215880,57973920,58362240,"[0.8444742589447913, 35.9684823423289, 24.9154...",1.629499,81.583000,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,8090,"[[1207790, 1207835], [1208385, 1208415], [1208...",14,"[1, 8, 21, 119, 184, 191, 313, 353, 365, 382, ...",49,165.102041,"[-8, -9, -8, -8, -8, -8, -9, -9, -9, -9, -9, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1740715,1800000,83554320,86400000,"[3.966666274879191, 7.803691105242437, 11.8754...",3.890374,182.872287,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,59285,"[[1740715, 1740800], [1742180, 1742270], [1742...",207,"[10, 303, 310, 437, 449, 462, 469, 476, 484, 4...",748,79.258021,"[-6, -6, -6, -6, -6, -6, -5, -5, -5, -6, -5, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,64970,113710,3118560,5458080,"[21.461368916630207, 53.33695817890708, 69.304...",10.189721,172.492812,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,48740,"[[64970, 65050], [66680, 66755], [67710, 67740...",199,"[8, 275, 350, 551, 619, 642, 649, 659, 669, 67...",572,85.209790,"[-5, -5, -4, -4, -5, -4, -4, -5, -5, -4, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1015995,1071650,48767760,51439200,"[9.416056073970946, 28.207963051592127, 40.983...",10.189721,194.741365,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,55655,"[[1015995, 1016070], [1017820, 1017895], [1019...",203,"[6, 13, 297, 371, 665, 736, 757, 944, 971, 978...",709,78.497884,"[-2, -1, -1, -1, -1, -2, -1, -1, -2, -1, -2, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,1479420,1538630,71012160,73854240,"[35.69708754130259, 62.5539758266018, 45.36143...",10.189721,191.488972,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,59210,"[[1479420, 1479495], [1480620, 1480650], [1481...",221,"[6, 13, 243, 326, 342, 566, 596, 620, 628, 636...",825,71.769697,"[3, 3, 4, 4, 4, 3, 3, 2, 3, 3, 3, 2, 2, 3, 3, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,279405,317305,13411440,15230640,"[3.290829698243165, 4.017149096026861, 4.36750...",4.910507,160.571495,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,37900,"[[279405, 279735], [280905, 281300], [283150, ...",94,"[4, 12, 19, 27, 42, 50, 64, 312, 322, 330, 337...",268,141.417910,"[-2, -2, -1, -2, -1, -2, -1, -1, -2, -1, -2, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
79,391960,398450,18814080,19125600,"[2.1287592979018926, 4.138574440889828, 5.2615...",4.910507,64.761193,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,6490,"[[391960, 392040], [393455, 393890], [394245, ...",9,"[7, 14, 313, 321, 343, 357, 366, 374, 464, 554...",21,309.047619,"[2, 2, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
80,402835,438620,19336080,21053760,"[5.355641041130086, 17.40277914473961, 20.7160...",4.910507,157.202025,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,35785,"[[402835, 402870], [403190, 403265], [404415, ...",153,"[2, 76, 320, 389, 409, 586, 616, 637, 650, 659...",454,78.821586,"[-5, -5, -6, -7, -6, -6, -6, -6, -6, -5, -6, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
81,826420,864745,39668160,41507760,"[1.974171837079516, 1.539925229185384, 2.42539...",4.910507,147.871824,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,38325,"[[826420, 826520], [828030, 828155], [828175, ...",109,"[9, 16, 326, 354, 376, 385, 401, 414, 422, 431...",320,119.765625,"[10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 1...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [41]:
bpd['file'][0]

'/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-31/alsa/07-00-01-01.wav'

## inspect the bouts and curate them

#### visualize one bout

In [42]:
bpd.iloc[0]

start_ms                                                    1207790
end_ms                                                      1215880
start_sample                                               57973920
end_sample                                                 58362240
p_step            [0.8444742589447913, 35.9684823423289, 24.9154...
rms_p                                                      1.629499
peak_p                                                       81.583
bout_check                                                     True
file              /mnt/sphere/speech_bci/raw_data/s_b1253_21/202...
len_ms                                                         8090
syl_in            [[1207790, 1207835], [1208385, 1208415], [1208...
n_syl                                                            14
peaks_p           [1, 8, 21, 119, 184, 191, 313, 353, 365, 382, ...
n_peaks                                                          49
l_p_ratio                                       

In [43]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [80]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bpd.iloc[24])

    'data': [{'type': 'scatter',
              'uid': '69aabcb9-5ecb-4a7b-98e5-2…

In [45]:
bpd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram
0,1207790,1215880,57973920,58362240,"[0.8444742589447913, 35.9684823423289, 24.9154...",1.629499,81.583000,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,8090,"[[1207790, 1207835], [1208385, 1208415], [1208...",14,"[1, 8, 21, 119, 184, 191, 313, 353, 365, 382, ...",49,165.102041,"[-8, -9, -8, -8, -8, -8, -9, -9, -9, -9, -9, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1740715,1800000,83554320,86400000,"[3.966666274879191, 7.803691105242437, 11.8754...",3.890374,182.872287,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,59285,"[[1740715, 1740800], [1742180, 1742270], [1742...",207,"[10, 303, 310, 437, 449, 462, 469, 476, 484, 4...",748,79.258021,"[-6, -6, -6, -6, -6, -6, -5, -5, -5, -6, -5, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [46]:
bpd.reset_index(drop=True, inplace=True)

In [47]:
## Set confusing by default, will only be False once asserted bout/or not
bpd['confusing'] = True
bpd['bout_check'] = False

In [48]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [49]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bpd)

In [50]:
np.where(viz_bout.bouts_pd['bout_check']==True)[0].size

82

### save it

In [51]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 48000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function __main__.read_wav_chan(wav_path: str, chan_id: int = 0) -> tuple>,
 'file_order_fun': <function __main__.sess_file_id(f_path)>,
 'min_segment': 30,
 'min_silence': 2000,
 'min_bout': 5000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 120000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [52]:
### get the curated file path
##save to the curated file path
bouts_curated_file_path = os.path.join(bouts_folder, hparams['bout_curated_file'])

viz_bout.bouts_pd.to_pickle(bouts_curated_file_path)
logger.info('saved curated bout pandas to pickle {}'.format(bouts_curated_file_path))

2021-08-06 13:02:11,261 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-31/alsa/bouts_ceciestunepipe/bout_checked.pickle


In [ ]:
viz_bout.bouts_pd['file'][0]

In [ ]:
viz_bout.bouts_pd.head(5)